In [51]:
!pip install langchain langchain-google-genai duckduckgo-search youtube_search wikipedia langchainhub langchain_experimental

import os
import getpass

from langchain.agents import Tool, AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.agents import AgentExecutor
from langchain import hub, LLMMathChain
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description

In [52]:
#use your own API key, from (https://makersuite.google.com/), remember to use VPN if you are not in US
#put your key inside the apostrophe....
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

In [53]:
# setup model
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    convert_system_message_to_human=True,
    handle_parsing_errors=True,
    temperature=0.6,
    max_tokens= 200,
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    },
)

In [54]:
ddg_search = DuckDuckGoSearchAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

tools = [
    Tool(
        name="DuckDuckGo Search",
        func=ddg_search.run,
        description="Useful to browse information from the internet to know recent results and information you don't know. Then, tell user the result.",
    ),
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Useful for when you need to answer questions about very simple math. This tool is only for math questions and nothing else. Only input math expressions. Not supporting symbolic math.",
    ),
]

In [55]:
agent_prompt = hub.pull("mikechan/gemini")
agent_prompt.template

'Your name is “Kasper”. You are world class ai personal assistant computer that created by Mike Chan(A independent AI engineer). You are highly advanced intelligent. you are very helpful and good to provide information and solve users problem. Kasper is a AELM model (Auto Execution Language Model).\n\n“ Kasper” is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, “ Kasper” is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\n“ Kasper” is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, “ Kasper” is

In [56]:
prompt = agent_prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)
llm_with_stop = llm.bind(stop=["\nObservation"])

In [57]:
memory = ConversationBufferMemory(memory_key="chat_history")

In [58]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

In [60]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://python.langchain.com/docs/modules/tools/")
docs=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

llm = ChatGoogleGenerativeAI(model="gemini-pro",convert_system_message_to_human=True)

from langchain_community.vectorstores import FAISS
vector_db = FAISS.from_documents(documents[:15], OllamaEmbeddings())

In [50]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

retriever=vector_db.as_retriever()

from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langchain_tools","give difference between langchain tools and langchain chains")

from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)

tools=[wiki,arxiv,retriever_tool]

from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

# from langchain.agents import create_openai_tools_agent
# agent=create_openai_tools_agent(llm,tools,prompt)

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools,verbose=True)


agent_executor.invoke({"input":"tell me about langserve"})



> Entering new AgentExecutor chain...


c:\Users\arjun.sreekumar\AppData\Local\anaconda3\envs\rag_pipeline\lib\site-packages\langchain_google_genai\chat_models.py:308: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


ValueError: Unknown field for FunctionDeclaration: type

In [36]:
agent

{
  input: RunnableLambda(...),
  agent_scratchpad: RunnableLambda(...),
  chat_history: RunnableLambda(...)
}
| PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], partial_variables={'tools': "DuckDuckGo Search: Useful to browse information from the internet to know recent results and information you don't know. Then, tell user the result.\nCalculator: Useful for when you need to answer questions about very simple math. This tool is only for math questions and nothing else. Only input math expressions. Not supporting symbolic math.", 'tool_names': 'DuckDuckGo Search, Calculator'}, metadata={'lc_hub_owner': 'mikechan', 'lc_hub_repo': 'gemini', 'lc_hub_commit_hash': '5f76d149fc49238958280d2b2f438178d19c8d447251225bee2a08d9bf7656d3'}, template='Your name is “Kasper”. You are world class ai personal assistant computer that created by Mike Chan(A independent AI engineer). You are highly advanced intelligent. you are very helpful and good to provide information and

In [37]:
tools

[Tool(name='DuckDuckGo Search', description="Useful to browse information from the internet to know recent results and information you don't know. Then, tell user the result.", func=<bound method DuckDuckGoSearchAPIWrapper.run of DuckDuckGoSearchAPIWrapper(region='wt-wt', safesearch='moderate', time='y', max_results=5, backend='api', source='text')>),
 Tool(name='Calculator', description='Useful for when you need to answer questions about very simple math. This tool is only for math questions and nothing else. Only input math expressions. Not supporting symbolic math.', func=<bound method Chain.run of LLMMathChain(verbose=True, llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\

In [38]:
agent_executor.invoke({"input": "calculate this: The marked price of a chair is 20 percentage higher than its cost. The chair is sold at a discount of $90 on its marked price. After selling the chair, the percentage loss is 16%. Find the cost of the chair."})["output"]



> Entering new AgentExecutor chain...


c:\Users\arjun.sreekumar\AppData\Local\anaconda3\envs\rag_pipeline\lib\site-packages\langchain_google_genai\chat_models.py:308: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: Let x be the cost of the chair. 1.2x is the marked price. 1.2x - 90 is the selling price. 1 - (1.2x - 90) / x * 100 = 16. Solve for x.

> Entering new LLMMathChain chain...
Let x be the cost of the chair. 1.2x is the marked price. 1.2x - 90 is the selling price. 1 - (1.2x - 90) / x * 100 = 16. Solve for x.

c:\Users\arjun.sreekumar\AppData\Local\anaconda3\envs\rag_pipeline\lib\site-packages\langchain_google_genai\chat_models.py:308: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```text
(1 - (1.2 * x - 90) / x * 100) == 16
```
...numexpr.evaluate("(1 - (1.2 * x - 90) / x * 100) == 16")...


ValueError: LLMMathChain._evaluate("
(1 - (1.2 * x - 90) / x * 100) == 16
") raised error: 'x'. Please try again with a valid numerical expression